In [1]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [57]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet

In [20]:
data = pd.read_csv("data.csv.gz")

In [21]:
data.shape

(1620666, 37)

In [22]:
firm_characteristics = ['me', 'be_me', 'ope_be', 'at_gr1', 'ret_3_1']
macro_variables = [col for col in data.columns if col not in firm_characteristics][2:]

print("Firm Characteristics:", firm_characteristics)
print("Macro Variables:", macro_variables)

Firm Characteristics: ['me', 'be_me', 'ope_be', 'at_gr1', 'ret_3_1']
Macro Variables: ['TB3MS', 'GS1', 'GS5', 'GS10', 'GS30', 'BAA', 'AAA', 'UNRATE', 'ICSA', 'M2', 'FEDFUNDS', 'VIX', 'OIL', 'INDPRO', 'TCU', 'RPI', 'PAYEMS', 'CPI', 'CPICORE', 'PCEPI', 'term_spread', 'def_spread', 'inflation', 'core_inflation', 'real_ff', 'm2_growth', 'indpro_growth', 'rpi_growth', 'payems_growth', 'oil_ret']


In [23]:
for firm_char in firm_characteristics:
    for macro_var in macro_variables:
        new_col_name = f"{firm_char}_x_{macro_var}"
        data[new_col_name] = data[firm_char] * data[macro_var]

C:\Users\yimengs\AppData\Local\Temp\ipykernel_21272\3621462998.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col_name] = data[firm_char] * data[macro_var]
C:\Users\yimengs\AppData\Local\Temp\ipykernel_21272\3621462998.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col_name] = data[firm_char] * data[macro_var]
C:\Users\yimengs\AppData\Local\Temp\ipykernel_21272\3621462998.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

In [24]:
unscale_cols = ["eom", "ret_exc_lead1m"]
scale_cols = [c for c in data.columns if c not in unscale_cols]

In [25]:
data.head()

,eom,ret_exc_lead1m,me,be_me,ope_be,at_gr1,ret_3_1,TB3MS,GS1,GS5,...,ret_3_1_x_term_spread,ret_3_1_x_def_spread,ret_3_1_x_inflation,ret_3_1_x_core_inflation,ret_3_1_x_real_ff,ret_3_1_x_m2_growth,ret_3_1_x_indpro_growth,ret_3_1_x_rpi_growth,ret_3_1_x_payems_growth,ret_3_1_x_oil_ret
0,1990-02-28,0.065049,20.447000,0.626693,0.165990,0.479253,-0.176471,7.74,8.11,8.42,...,-0.128824,-0.162353,-0.000691,-0.000800,-1.445829,-0.000690,-0.001595,-0.000628,-0.000387,0.009097
1,1990-02-28,-0.028602,40.168125,0.687510,0.219076,0.196787,-0.076923,7.74,8.11,8.42,...,-0.056154,-0.070769,-0.000301,-0.000349,-0.630233,-0.000301,-0.000695,-0.000274,-0.000169,0.003965
2,1990-02-28,0.072644,5.634938,0.876319,0.104293,0.034325,0.078947,7.74,8.11,8.42,...,0.057632,0.072632,0.000309,0.000358,0.646818,0.000309,0.000714,0.000281,0.000173,-0.004070
3,1990-02-28,0.053698,5012.359500,0.291460,0.460676,0.050157,-0.051596,7.74,8.11,8.42,...,-0.037665,-0.047468,-0.000202,-0.000234,-0.422727,-0.000202,-0.000466,-0.000184,-0.000113,0.002660
4,1990-02-28,0.071290,175.705125,1.416902,0.010831,0.003274,-0.157895,7.74,8.11,8.42,...,-0.115263,-0.145263,-0.000618,-0.000716,-1.293637,-0.000617,-0.001427,-0.000562,-0.000347,0.008139


In [51]:
# 1. Sort by date and reset index so index == row position
data_n = data.sort_values("eom").reset_index(drop=True)

# 2. Find the row position where test period begins
cutoff_date = "2020-01-31"
test_idx = data_n.index[data_n["eom"] >= cutoff_date][0]  # now this is a position

# 3. Build train/test arrays directly from slices (no X_all / y_all)
X_train = data_n.loc[:test_idx-1, scale_cols].to_numpy(dtype="float32")
y_train = data_n.loc[:test_idx-1, "ret_exc_lead1m"].to_numpy(dtype="float32")

X_test  = data_n.loc[test_idx:, scale_cols].to_numpy(dtype="float32")
y_test  = data_n.loc[test_idx:, "ret_exc_lead1m"].to_numpy(dtype="float32")



In [52]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [54]:

from sklearn.linear_model import ElasticNetCV


# -----------------------------
# 4. Time-series Elastic Net with modest CV (to save RAM)
# -----------------------------

# Initialize TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=3)  # fewer splits = less memory and time

# Initialize ElasticNetCV with different alphas and l1_ratio (balance between Lasso and Ridge)
elasticnet_ts = ElasticNetCV(
    alphas=np.logspace(-7, -2, 100),  # Regularization strengths
    l1_ratio=[0.01, 0.02, 0.03, 0.04, 0.05],  # Ratio of Lasso to Ridge (0 = Ridge, 1 = Lasso)
    cv=tscv,
    max_iter=10000,  # Maximum iterations for the solver
    n_jobs=1,  # Number of CPUs to use for parallelization
    random_state=42
)

# Fit the ElasticNet model
elasticnet_ts.fit(X_train, y_train)

# Print the best alpha and l1_ratio values
print("Best alpha (regularization strength) =", elasticnet_ts.alpha_)
print("Best l1_ratio (Lasso vs Ridge balance) =", elasticnet_ts.l1_ratio_)

# Print the number of non-zero coefficients (sparsity)
print("Number of nonzero coefficients:", np.sum(elasticnet_ts.coef_ != 0))


C:\Users\yimengs\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.246e+00, tolerance: 8.556e-01
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\yimengs\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.698e+01, tolerance: 8.556e-01
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\yimengs\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the feature

Best alpha (regularization strength) = 0.01
Best l1_ratio (Lasso vs Ridge balance) = 0.05
Number of nonzero coefficients: 21


In [56]:
coefs = elasticnet_ts.coef_
nonzero = [(name, coef) for name, coef in zip(scale_cols, coefs) if coef != 0]
survive=[]
for name, coef in nonzero:
    print(f"{name:25s}  {coef: .5f}")
    survive.append(name)

GS5                        -0.01086
UNRATE                      0.00495
ICSA                        0.00544
VIX                        -0.00266
OIL                        -0.01102
term_spread                -0.00568
core_inflation             -0.00320
m2_growth                  -0.00158
indpro_growth               0.00518
rpi_growth                  0.00111
payems_growth              -0.00013
oil_ret                    -0.00198
me_x_indpro_growth          0.00024
me_x_payems_growth          0.00006
be_me_x_UNRATE              0.00301
be_me_x_ICSA                0.00330
be_me_x_OIL                -0.00245
be_me_x_inflation           0.00009
be_me_x_m2_growth          -0.00171
be_me_x_indpro_growth       0.00038
be_me_x_payems_growth      -0.00261


In [59]:

# Assume you already ran something like:
# enet_ts = ElasticNetCV(...).fit(X_train, y_train)

en_final = ElasticNet(
    alpha=elasticnet_ts.alpha_,
    l1_ratio=elasticnet_ts.l1_ratio_,
    max_iter=5000
)

en_final.fit(X_train, y_train)

y_pred_test = en_final.predict(X_test)

ic_test  = np.corrcoef(y_test, y_pred_test)[0, 1]
mse_test = np.mean((y_test - y_pred_test)**2)
r2_test  = 1 - mse_test / np.var(y_test)

print("Elastic Net Test IC:", ic_test)
print("Elastic Net Test MSE:", mse_test)
print("Elastic Net Test R^2:", r2_test)


Elastic Net Test IC: 0.12183929968070947
Elastic Net Test MSE: 0.05635891
Elastic Net Test R^2: -0.055223941802978516


In [60]:
keep_cols = ["eom", "ret_exc_lead1m"] + survive
data_reduced = data_n[keep_cols]


In [62]:
data_reduced.to_csv("data_new.csv.gz", index=False, compression="gzip")